# Validation of islanded mode equivalence

## Import and helpers
Import core packages for the toy model.

In [1]:
import logging
import numpy as np
import pandas as pd
import pypsa
import plotly.graph_objects as go
logger = logging.getLogger(__name__)

Helpers

In [76]:
def extract_capacity(n: pypsa.Network) -> pd.DataFrame:
    """Extract a tidy table of optimal capacities for all extendable assets."""
    df = n.statistics.optimal_capacity(nice_names=False).reset_index()
    df.columns = ["component", "name", "capacity"]
    return df


def extract_energy_balance(
    n: pypsa.Network,
    bus_carrier="AC",
    ) -> pd.DataFrame:
    """Extract a tidy table of energy balance statistics."""
    df = n.statistics.energy_balance(nice_names=False, bus_carrier=bus_carrier)
    df.rename("energy", inplace=True)
    return df.reset_index()


def extract_energy_balance_time(
    n: pypsa.Network,
    bus_carrier="AC",
    ) -> pd.DataFrame:
    """Extract a tidy table of energy balance statistics."""
    df = n.statistics.energy_balance(nice_names=False, groupby_time=False, bus_carrier=bus_carrier)
    return df.reset_index()


def calculate_total_costs(n: pypsa.Network, exclude_grid: bool = False) -> float:
    """Calculate total costs (capex + opex), optionally excluding AC lines and DC links."""
    costs = pd.concat([n.statistics.capex(nice_names=False), n.statistics.opex(nice_names=False)], axis=1, keys=["capex", "opex"])
    if exclude_grid:
        costs = costs[~(
            ((costs.index.get_level_values("component") == "Line") &
             (costs.index.get_level_values("carrier") == "AC")) |
            ((costs.index.get_level_values("component") == "Link") &
             (costs.index.get_level_values("carrier") == "DC"))
        )]
    return costs.fillna(0).sum().sum()


def create_bar_plot(
    pivot_df, 
    x_col, 
    y_cols, 
    labels, 
    title, 
    yaxis_title,
    height=600,
    width=800,
):
    """Create a grouped bar plot with multiple scenarios."""
    fig = go.Figure()
    for y_col, label in zip(y_cols, labels):
        fig.add_trace(go.Bar(
            x=pivot_df[x_col], 
            y=pivot_df[y_col], 
            name=label,
        ))
    fig.update_layout(
        title=title,
        xaxis_title=x_col.title(),
        yaxis_title=yaxis_title,
        barmode="group",
        hovermode="x unified",
        template="plotly_white",
        height=height,
        width=width,
    )
    return fig


def create_stacked_bar_plot(
    df, 
    scenario_col, 
    value_col, 
    carrier_col, 
    carrier_colors, 
    scenario_labels, 
    title, 
    yaxis_title,
    height=800,
    width=600,
):
    """Create a stacked bar plot with carriers colored by carrier_colors dict.
    
    Positive values are stacked upward (generation), negative values downward (demand).
    
    Parameters
    ----------
    df : pd.DataFrame
        Long-form dataframe with scenario, carrier, and value columns
    scenario_col : str
        Column name containing scenario identifiers
    value_col : str
        Column name containing values to plot
    carrier_col : str
        Column name containing carrier names
    carrier_colors : dict
        Dictionary mapping carrier names to colors
    scenario_labels : dict
        Dictionary mapping scenario identifiers to display labels
    title : str
        Plot title
    yaxis_title : str
        Y-axis label
    """
    fig = go.Figure()
    
    # Define the order of scenarios
    scenario_order = ["grid", "maxpu0", "removed", "outage_summer", "outage_winter", "stochastic"]
    
    # Get unique carriers
    carriers = df[carrier_col].unique()
    
    # Create a complete dataframe with all scenario-carrier combinations
    # Group by scenario and carrier to aggregate any duplicates
    df_grouped = df.groupby([scenario_col, carrier_col], as_index=False)[value_col].sum()
    
    # Separate positive and negative values
    for carrier in carriers:
        carrier_data = df_grouped[df_grouped[carrier_col] == carrier].copy()
        
        # Split into positive and negative
        positive_data = carrier_data[carrier_data[value_col] >= 0].copy()
        negative_data = carrier_data[carrier_data[value_col] < 0].copy()
        
        color = carrier_colors.get(carrier, "#cccccc")
        
        # Add positive values (generation)
        if not positive_data.empty:
            # Ensure all scenarios are present
            x_vals = []
            y_vals = []
            for scenario in scenario_order:
                x_vals.append(scenario_labels.get(scenario, scenario))
                matching = positive_data[positive_data[scenario_col] == scenario]
                if not matching.empty:
                    y_vals.append(matching[value_col].values[0])
                else:
                    y_vals.append(0)
            
            fig.add_trace(go.Bar(
                name=carrier,
                x=x_vals,
                y=y_vals,
                marker=dict(color=color),
                legendgroup=carrier,
                showlegend=True,
            ))
        
        # Add negative values (demand)
        if not negative_data.empty:
            # Ensure all scenarios are present
            x_vals = []
            y_vals = []
            for scenario in scenario_order:
                x_vals.append(scenario_labels.get(scenario, scenario))
                matching = negative_data[negative_data[scenario_col] == scenario]
                if not matching.empty:
                    y_vals.append(matching[value_col].values[0])
                else:
                    y_vals.append(0)
            
            fig.add_trace(go.Bar(
                name=carrier,
                x=x_vals,
                y=y_vals,
                marker=dict(color=color),
                legendgroup=carrier,
                showlegend=True,
            ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Scenario",
        yaxis_title=yaxis_title,
        barmode="relative",  # This stacks positive up and negative down
        hovermode="x unified",
        template="plotly_white",
        height=height,
        width=width,
    )
    
    return fig


def create_stacked_area_plot(
    df, 
    time_col,
    scenario_col, 
    value_col, 
    carrier_col, 
    carrier_colors, 
    scenario_labels, 
    title, 
    yaxis_title,
    height=600,
    width=2000,
):
    """Create stacked area plots over time for multiple scenarios in subplots.
    
    Positive values are stacked upward (generation), negative values downward (demand).
    
    Parameters
    ----------
    df : pd.DataFrame
        Long-form dataframe with time, scenario, carrier, and value columns
    time_col : str
        Column name containing time/snapshot identifiers
    scenario_col : str
        Column name containing scenario identifiers
    value_col : str
        Column name containing values to plot
    carrier_col : str
        Column name containing carrier names
    carrier_colors : dict
        Dictionary mapping carrier names to colors
    scenario_labels : dict
        Dictionary mapping scenario identifiers to display labels
    title : str
        Plot title
    yaxis_title : str
        Y-axis label
    height : int
        Height of the figure in pixels
    width : int
        Width of the figure in pixels
    """
    from plotly.subplots import make_subplots
    
    # Define the order of scenarios
    scenario_order = ["grid", "maxpu0", "removed", "outage_summer", "outage_winter", "stochastic"]
    n_scenarios = len(scenario_order)
    
    # Create subplots - one column per scenario
    fig = make_subplots(
        rows=1, 
        cols=n_scenarios,
        subplot_titles=[scenario_labels.get(s, s) for s in scenario_order],
        shared_yaxes=True,
        horizontal_spacing=0.02,
    )
    
    # Get unique carriers
    carriers = sorted([c for c in df[carrier_col].unique() if c in carrier_colors])
    
    # Ensure time column is datetime type for consistent merging
    df[time_col] = pd.to_datetime(df[time_col])
    
    # Get all unique time points across all scenarios (sorted)
    all_times = sorted(df[time_col].unique())
    
    # Process each scenario
    for col_idx, scenario in enumerate(scenario_order, start=1):
        scenario_data = df[df[scenario_col] == scenario].copy()
        
        if scenario_data.empty:
            continue
        
        # Process each carrier
        for carrier in carriers:
            carrier_data = scenario_data[scenario_data[carrier_col] == carrier].copy()
            
            if carrier_data.empty:
                continue
            
            # Create a complete time series with all time points
            # This ensures proper stacking and hover alignment
            complete_series = pd.DataFrame({time_col: pd.to_datetime(all_times)})
            carrier_data = complete_series.merge(
                carrier_data[[time_col, value_col]], 
                on=time_col, 
                how='left'
            ).fillna(0)
            
            # Sort by time
            carrier_data = carrier_data.sort_values(time_col)
            
            # Get values
            x_vals = carrier_data[time_col]
            y_vals = carrier_data[value_col]
            
            color = carrier_colors.get(carrier, "#cccccc")
            showlegend = True # (col_idx == 1)
            
            # Split into positive and negative by creating separate series
            y_positive = y_vals.where(y_vals >= 0, 0)
            y_negative = y_vals.where(y_vals < 0, 0)
            
            # Add positive trace if there are any positive values
            if (y_positive != 0).any():
                fig.add_trace(
                    go.Scatter(
                        x=x_vals,
                        y=y_positive,
                        name=carrier,
                        mode='lines',
                        line=dict(width=0),
                        stackgroup='positive',
                        fillcolor=color,
                        legendgroup=carrier,
                        showlegend=showlegend,
                        hovertemplate=f'{carrier}<br>%{{y:.2f}}<extra></extra>',
                    ),
                    row=1, col=col_idx
                )
            
            # Add negative trace if there are any negative values
            if (y_negative != 0).any():
                fig.add_trace(
                    go.Scatter(
                        x=x_vals,
                        y=y_negative,
                        name=carrier,
                        mode='lines',
                        line=dict(width=0),
                        stackgroup='negative',
                        fillcolor=color,
                        legendgroup=carrier,
                        showlegend=showlegend,
                        hovertemplate=f'{carrier}<br>%{{y:.2f}}<extra></extra>',
                    ),
                    row=1, col=col_idx
                )
    
    # Update layout
    fig.update_layout(
        height=height,
        width=width,
        template="plotly_white",
        hovermode="x unified",
        title_text=title,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.01
        )
    )
    
    # Update all x-axes
    for i in range(1, n_scenarios + 1):
        fig.update_xaxes(title_text="Time", row=1, col=i)
    
    # Update y-axis only for the first subplot

    fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
    fig.update_yaxes(title_text=yaxis_title, row=1, col=1)    
    
    return fig

## Data prep
Load the base network, adjust loads, and add load-shedding generators.

In [3]:
n_elec_path = "data/networks/base_s_50_elec_.nc"
solver_name = "gurobi"
solver_options = {"OutputFlag": 0}  # minimize solver log output

# Load network
n = pypsa.Network(n_elec_path)

# Base styling and carriers
n.carriers.loc["", "color"] = "#aaaaaa"
n.add("Carrier", "load shedding", color="#aa0000")

# Increase load
n.loads_t.p_set *= 1.7

# Add load-shedding generators
spatial_nodes = n.buses.index.tolist()
ls_names = [f"{bus} load shedding" for bus in spatial_nodes]

n.add(
    "Generator",
    pd.Index(ls_names),
    bus=pd.Series(spatial_nodes, index=ls_names),
    p_nom_extendable=True,
    p_nom_max=np.inf,
    capital_cost=0.1,
    marginal_cost=10000.0,
    carrier="load shedding",
)

INFO:pypsa.network.io:New version 1.0.5 available! (Current: 1.0.4)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, carriers, generators, lines, links, loads, storage_units, stores, sub_networks


## Optimisation
Prepare scenario copies for baseline and islanded configurations. Run optimisations for baseline, s_max_pu = 0 / p_max_pu = 0, and fully islanded cases.

In [4]:
print("Running baseline optimisation...")
n1 = n.copy()
n1.optimize(solver_name=solver_name, solver_options=solver_options)

Running baseline optimisation...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 420.31it/s]
INFO:linopy.io: Writing time: 0.35s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

INFO:gurobipy:Reading time = 0.07 seconds


obj: 53897 rows, 25873 columns, 107777 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 107777 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 9.66e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [5]:
print("Running islanded optimisation (s_max_pu = 0 and p_max_pu = 0)...")
n2 = n.copy()
if not n.lines.empty:
    n2.lines["s_max_pu"] = 0
if not n2.links.empty:
    dc_links = n2.links[n2.links.carrier == "DC"].index
    n2.links.loc[dc_links, ["p_max_pu", "p_min_pu"]] = 0

n2.optimize(solver_name=solver_name, solver_options=solver_options)

Running islanded optimisation (s_max_pu = 0 and p_max_pu = 0)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 557.10it/s]
INFO:linopy.io: Writing time: 0.28s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

INFO:gurobipy:Reading time = 0.06 seconds


obj: 53897 rows, 25873 columns, 100769 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 100769 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 1.28e+11
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [6]:
print("Running islanded optimisation (lines and links removed)...")
n3 = n.copy()
if not n.lines.empty:
    n3.remove("Line", n3.lines.index)
if not n3.links.empty:
    n3.remove("Link", n3.links[n3.links.carrier == "DC"].index)

n3.optimize(solver_name=solver_name, solver_options=solver_options)

Running islanded optimisation (lines and links removed)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 12/12 [00:00<00:00, 511.45it/s]
INFO:linopy.io: Writing time: 0.27s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

INFO:gurobipy:Reading time = 0.05 seconds


obj: 44499 rows, 21798 columns, 80475 nonzeros


INFO:gurobipy:obj: 44499 rows, 21798 columns, 80475 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 21798 primals, 44499 duals
Objective: 1.28e+11
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [7]:
print("Running islanded optimisation (AC lines and DC links zero availability in summer: Q2+Q3)...")
n4 = n.copy()

snapshots = n4.snapshots
quarters = np.array_split(snapshots, 4)
q1, q2, q3, q4 = quarters
outage_summer = q2.union(q3)

if not n4.lines.empty:
    ac_lines = n4.lines.index[n4.lines.carrier == "AC"]
    lines_avail = pd.DataFrame(index=snapshots, columns=ac_lines, dtype=float)
    lines_avail[:] = n4.lines.loc[ac_lines, "s_max_pu"]
    lines_avail.loc[outage_summer, :] = 0
    n4.lines_t.s_max_pu = lines_avail

if not n4.links.empty:
    dc_links = n4.links.index[n4.links.carrier == "DC"]
    links_avail = pd.DataFrame(index=snapshots, columns=dc_links, dtype=float)
    links_avail[:] = n4.links.loc[dc_links, "p_max_pu"]
    links_avail.loc[outage_summer, :] = 0
    n4.links_t.p_max_pu = links_avail
    n4.links_t.p_min_pu = -links_avail

n4.optimize(solver_name=solver_name, solver_options=solver_options)

Running islanded optimisation (AC lines and DC links zero availability in summer: Q2+Q3)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 413.80it/s]
INFO:linopy.io: Writing time: 0.36s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

INFO:gurobipy:Reading time = 0.11 seconds


obj: 53897 rows, 25873 columns, 104681 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 104681 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 1.07e+11
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [8]:
print("Running islanded optimisation (AC lines and DC links zero availability in winter: Q1+Q4)...")
n5 = n.copy()

snapshots = n5.snapshots
quarters = np.array_split(snapshots, 4)
q1, q2, q3, q4 = quarters
outage_winter = q1.union(q4)

if not n5.lines.empty:
    ac_lines = n5.lines.index[n5.lines.carrier == "AC"]
    lines_avail = pd.DataFrame(index=snapshots, columns=ac_lines, dtype=float)
    lines_avail[:] = n5.lines.loc[ac_lines, "s_max_pu"]
    lines_avail.loc[outage_winter, :] = 0
    n5.lines_t.s_max_pu = lines_avail

if not n5.links.empty:
    dc_links = n5.links.index[n5.links.carrier == "DC"]
    links_avail = pd.DataFrame(index=snapshots, columns=dc_links, dtype=float)
    links_avail[:] = n5.links.loc[dc_links, "p_max_pu"]
    links_avail.loc[outage_winter, :] = 0
    n5.links_t.p_max_pu = links_avail
    n5.links_t.p_min_pu = -links_avail

n5.optimize(solver_name=solver_name, solver_options=solver_options)

Running islanded optimisation (AC lines and DC links zero availability in winter: Q1+Q4)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 466.67it/s]
INFO:linopy.io: Writing time: 0.33s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

INFO:gurobipy:Reading time = 0.07 seconds


obj: 53897 rows, 25873 columns, 104681 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 104681 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 1.15e+11
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [26]:
n6.lines_t.s_max_pu

scenario                 summer_outage                                         \
name                (summer_outage, 0) (summer_outage, 1) (summer_outage, 10)   
snapshot                                                                        
2013-01-01 00:00:00                NaN                NaN                 NaN   
2013-01-16 05:00:00                NaN                NaN                 NaN   
2013-01-31 10:00:00                NaN                NaN                 NaN   
2013-02-15 15:00:00                NaN                NaN                 NaN   
2013-03-02 20:00:00                NaN                NaN                 NaN   
2013-03-18 01:00:00                NaN                NaN                 NaN   
2013-04-02 06:00:00                0.0                0.0                 0.0   
2013-04-17 11:00:00                0.0                0.0                 0.0   
2013-05-02 16:00:00                0.0                0.0                 0.0   
2013-05-17 21:00:00                0.0                0.0                 0.0   
2013-06-02 02:00:00                0.0                0.0                 0.0   
2013-06-17 07:00:00                0.0                0.0                 0.0   
2013-07-02 12:00:00                0.0                0.0                 0.0   
2013-07-17 17:00:00                0.0                0.0                 0.0   
2013-08-01 22:00:00                0.0                0.0                 0.0   
2013-08-17 03:00:00                0.0                0.0                 0.0   
2013-09-01 08:00:00                0.0                0.0                 0.0   
2013-09-16 13:00:00                0.0                0.0                 0.0   
2013-10-01 18:00:00                NaN                NaN                 NaN   
2013-10-16 23:00:00                NaN                NaN                 NaN   
2013-11-01 04:00:00                NaN                NaN                 NaN   
2013-11-16 09:00:00                NaN                NaN                 NaN   
2013-12-01 14:00:00                NaN                NaN                 NaN   
2013-12-16 19:00:00                NaN                NaN                 NaN   

scenario                                                     \
name                (summer_outage, 11) (summer_outage, 12)   
snapshot                                                      
2013-01-01 00:00:00                 NaN                 NaN   
2013-01-16 05:00:00                 NaN                 NaN   
2013-01-31 10:00:00                 NaN                 NaN   
2013-02-15 15:00:00                 NaN                 NaN   
2013-03-02 20:00:00                 NaN                 NaN   
2013-03-18 01:00:00                 NaN                 NaN   
2013-04-02 06:00:00                 0.0                 0.0   
2013-04-17 11:00:00                 0.0                 0.0   
2013-05-02 16:00:00                 0.0                 0.0   
2013-05-17 21:00:00                 0.0                 0.0   
2013-06-02 02:00:00                 0.0                 0.0   
2013-06-17 07:00:00                 0.0                 0.0   
2013-07-02 12:00:00                 0.0                 0.0   
2013-07-17 17:00:00                 0.0                 0.0   
2013-08-01 22:00:00                 0.0                 0.0   
2013-08-17 03:00:00                 0.0                 0.0   
2013-09-01 08:00:00                 0.0                 0.0   
2013-09-16 13:00:00                 0.0                 0.0   
2013-10-01 18:00:00                 NaN                 NaN   
2013-10-16 23:00:00                 NaN                 NaN   
2013-11-01 04:00:00                 NaN                 NaN   
2013-11-16 09:00:00                 NaN                 NaN   
2013-12-01 14:00:00                 NaN                 NaN   
2013-12-16 19:00:00                 NaN                 NaN   

scenario                                                     \
name                (summer_outage, 13) (summer

In [53]:
print("Running stochastic optimisation (summer vs winter outage scenarios with 50/50 probability)...")
n6 = n.copy()

snapshots = n6.snapshots
quarters = np.array_split(snapshots, 4)
q1, q2, q3, q4 = quarters
t_summer = q2.union(q3)
t_winter = q1.union(q4)

# Get AC lines and DC links before setting scenarios
ac_lines = n6.lines.index[n6.lines.carrier == "AC"] if not n6.lines.empty else []
dc_links = n6.links.index[n6.links.carrier == "DC"] if not n6.links.empty else []

# Set up stochastic scenarios - this broadcasts all data across scenarios
n6.set_scenarios({"summer_outage": 0.5, "winter_outage": 0.5})

# Now modify the scenario-specific parameters
# After set_scenarios(), we access data using (scenario, component_name) tuples
if len(ac_lines) > 0:
    # For summer_outage scenario: set AC lines to 0 during summer
    for line in ac_lines:
        n6.lines_t.s_max_pu.loc[t_summer, ("summer_outage", line)] = 0.0
        n6.lines_t.s_max_pu.loc[t_winter, ("summer_outage", line)] = n6.lines.loc[("summer_outage", line), "s_max_pu"]
    # For winter_outage scenario: set AC lines to 0 during winter
    for line in ac_lines:
        n6.lines_t.s_max_pu.loc[t_winter, ("winter_outage", line)] = 0.0
        n6.lines_t.s_max_pu.loc[t_summer, ("winter_outage", line)] = n6.lines.loc[("winter_outage", line), "s_max_pu"]

if len(dc_links) > 0:
    # For summer_outage scenario: set DC links to 0 during summer
    for link in dc_links:
        n6.links_t.p_max_pu.loc[t_summer, ("summer_outage", link)] = 0.0
        n6.links_t.p_min_pu.loc[t_summer, ("summer_outage", link)] = 0.0
        n6.links_t.p_max_pu.loc[t_winter, ("summer_outage", link)] = n6.links.loc[("summer_outage", link), "p_max_pu"]
        n6.links_t.p_min_pu.loc[t_winter, ("summer_outage", link)] = n6.links.loc[("summer_outage", link), "p_min_pu"]
    # For winter_outage scenario: set DC links to 0 during winter
    for link in dc_links:
        n6.links_t.p_max_pu.loc[t_winter, ("winter_outage", link)] = 0.0
        n6.links_t.p_min_pu.loc[t_winter, ("winter_outage", link)] = 0.0
        n6.links_t.p_max_pu.loc[t_summer, ("winter_outage", link)] = n6.links.loc[("winter_outage", link), "p_max_pu"]
        n6.links_t.p_min_pu.loc[t_summer, ("winter_outage", link)] = n6.links.loc[("winter_outage", link), "p_min_pu"]

Running stochastic optimisation (summer vs winter outage scenarios with 50/50 probability)...


/tmp/ipykernel_85244/2710176597.py:26: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_85244/2710176597.py:26: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_85244/2710176597.py:26: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_85244/2710176597.py:26: Perform

In [54]:
# Note: Dual assignment fails with multi-indexed stochastic scenarios
# Catch the error since the solution is already assigned before dual assignment
try:
    n6.optimize(solver_name=solver_name, solver_options=solver_options, assign_all_duals=False)
except ValueError as e:
    if "cannot reindex on an axis with duplicate labels" in str(e):
        print("Optimization succeeded - solution assigned (dual assignment skipped due to multi-index)")
        print(f"Objective value: {n6.objective / 1e9:.3f} billion EUR")
    else:
        raise

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 403.53it/s]
INFO:linopy.io: Writing time: 0.4s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


INFO:gurobipy:Set parameter WLSSecret


...

INFO:gurobipy:Reading time = 0.14 seconds


obj: 112258 rows, 51122 columns, 255010 nonzeros


INFO:gurobipy:obj: 112258 rows, 51122 columns, 255010 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 51122 primals, 112258 duals
Objective: 1.17e+11
Solver model: available
Solver message: 2



Optimization succeeded - solution assigned (dual assignment skipped due to multi-index)
Objective value: 116.750 billion EUR


## Metrics
Metrics for validation

In [65]:
total_costs = pd.DataFrame({
    "scenario": [
        "Grid exists",
        "Grid outage (full year, s_max_pu=0)",
        "Grid outage (full year, lines removed)",
        "Grid outage (Summer)",
        "Grid outage (Winter)",
        # "Grid outage (Stochastic: Summer/Winter)",
    ],
    "total_cost": [
        np.round(calculate_total_costs(n1) / 1e9, 3),
        np.round(calculate_total_costs(n2, exclude_grid=False) / 1e9, 3),
        np.round(calculate_total_costs(n3, exclude_grid=True) / 1e9, 3),
        np.round(calculate_total_costs(n4, exclude_grid=False) / 1e9, 3),
        np.round(calculate_total_costs(n5, exclude_grid=False) / 1e9, 3),
        # np.round(calculate_total_costs(n6, exclude_grid=False) / 1e9, 3),
    ],
})
print("\nTotal annual costs by scenario:")
print(total_costs.to_string(index=False))


Total annual costs by scenario:
                              scenario  total_cost
                           Grid exists     324.880
   Grid outage (full year, s_max_pu=0)     356.123
Grid outage (full year, lines removed)     347.140
                  Grid outage (Summer)     334.888
                  Grid outage (Winter)     343.492


In [63]:
# mp_diff = n3.buses_t.marginal_price - n2.buses_t.marginal_price
# mp_diff_mean = mp_diff.abs().mean()
# print("\nMean absolute marginal price difference:")
# print(mp_diff_mean.head())

## Plots
Optimal capacities and energy balances across scenarios.

In [66]:
cap1 = extract_capacity(n1)
eb1 = extract_energy_balance(n1)
eb1_h2 = extract_energy_balance(n1, bus_carrier="H2")

cap2 = extract_capacity(n2)
eb2 = extract_energy_balance(n2)
eb2_h2 = extract_energy_balance(n2, bus_carrier="H2")

cap3 = extract_capacity(n3)
eb3 = extract_energy_balance(n3)
eb3_h2 = extract_energy_balance(n3, bus_carrier="H2")

cap4 = extract_capacity(n4)
eb4 = extract_energy_balance(n4)
eb4_h2 = extract_energy_balance(n4, bus_carrier="H2")

cap5 = extract_capacity(n5)
eb5 = extract_energy_balance(n5)
eb5_h2 = extract_energy_balance(n5, bus_carrier="H2")

# cap6 = extract_capacity(n6)
# eb6 = extract_energy_balance(n6)
# eb6_h2 = extract_energy_balance(n6, bus_carrier="H2")

In [71]:
capacities = pd.concat([
    cap1.assign(scenario="grid"),
    cap2.assign(scenario="maxpu0"),
    cap3.assign(scenario="removed"),
    cap4.assign(scenario="outage_summer"),
    cap5.assign(scenario="outage_winter"),
    # cap6.assign(scenario="stochastic"),
], ignore_index=True)

# Convert capacity from MW to GW for better readability
capacities["capacity_GW"] = capacities["capacity"] / 1e3

capacities_pivot = capacities.pivot_table(
    index=["component", "name"],
    columns="scenario",
    values="capacity_GW",
    fill_value=0,
).reset_index()

# Map name to carrier and color
name_to_carrier = pd.concat([
    n.generators["carrier"],
    n.storage_units["carrier"],
    n.stores["carrier"]
])

fig_cap = create_bar_plot(
    capacities_pivot,
    "name",  # x_col: column to use for x-axis (component names)
    ["grid", "maxpu0", "removed", "outage_summer", "outage_winter"],  # y_cols: scenario columns to plot
    [
        "Grid exists",
        "Grid outage (full year, s_max_pu=0)",
        "Grid outage (full year, lines removed)",
        "Grid outage (Summer)",
        "Grid outage (Winter)",
    ],  # labels: display names for each scenario
    "Optimal capacity",  # title
    "Installed capacity (GW)",  # yaxis_title
    height=600,
    width=1800,
)
fig_cap.show()

In [73]:
energy_balances = pd.concat([
    eb1.assign(scenario="grid"),
    eb2.assign(scenario="maxpu0"),
    eb3.assign(scenario="removed"),
    eb4.assign(scenario="outage_summer"),
    eb5.assign(scenario="outage_winter"),
    # eb6.assign(scenario="stochastic"),
], ignore_index=True)

# Convert energy from MWh to TWh for better readability
energy_balances["energy_TWh"] = energy_balances["energy"] / 1e6

# Map carriers to colors from n.carriers
carrier_color_map = {carrier: n.carriers.loc[carrier, "color"] 
                     for carrier in energy_balances["carrier"].unique() 
                     if carrier in n.carriers.index}

# Define scenario labels
scenario_labels = {
    "grid": "Grid exists",
    "maxpu0": "Grid outage (full year, s_max_pu=0)",
    "removed": "Grid outage (full year, lines removed)",
    "outage_summer": "Grid outage (Summer)",
    "outage_winter": "Grid outage (Winter)",
    # "stochastic": "Stochastic (summer vs winter)",
}

fig_eb_stacked = create_stacked_bar_plot(
    energy_balances,
    scenario_col="scenario",
    value_col="energy_TWh",
    carrier_col="carrier",
    carrier_colors=carrier_color_map,
    scenario_labels=scenario_labels,
    title="Energy balance by scenario",
    yaxis_title="Energy (TWh)",
    width=1000,
)
fig_eb_stacked.show()

In [77]:
# Extract time series energy balances with groupby_time=False
energy_balances_ts = pd.concat([
    extract_energy_balance_time(n1, bus_carrier="H2").assign(scenario="grid"),
    extract_energy_balance_time(n2, bus_carrier="H2").assign(scenario="maxpu0"),
    extract_energy_balance_time(n3, bus_carrier="H2").assign(scenario="removed"),
    extract_energy_balance_time(n4, bus_carrier="H2").assign(scenario="outage_summer"),
    extract_energy_balance_time(n5, bus_carrier="H2").assign(scenario="outage_winter"),
    extract_energy_balance_time(n6, bus_carrier="H2").assign(scenario="stochastic"),
], ignore_index=True)

# The data comes in wide format with snapshots as columns
# We need to reshape it to long format for the plotting function
id_vars = [col for col in energy_balances_ts.columns if col not in n1.snapshots and col != 'scenario']
energy_balances_ts_long = energy_balances_ts.melt(
    id_vars=id_vars + ['scenario'],
    var_name='snapshot',
    value_name='energy'
)

# Convert energy from MWh to GWh for better readability
energy_balances_ts_long["energy_GWh"] = energy_balances_ts_long["energy"] / 1e3

fig_eb_area = create_stacked_area_plot(
    energy_balances_ts_long,
    time_col="snapshot",
    scenario_col="scenario",
    value_col="energy_GWh",
    carrier_col="carrier",
    carrier_colors=carrier_color_map,
    scenario_labels=scenario_labels,
    title="Energy balance over time by scenario",
    yaxis_title="Power (GWh per timestep)",
    width=2400,
)
fig_eb_area.show()
